In [83]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sklearn
from sklearn.linear_model import LinearRegression
from copy import deepcopy

In [84]:
# !wget https://gitlab.com/evgeny.zavalnyuk/sirius_ml_labs/-/raw/main/ps1/sales.txt
sales_path='sales.txt'
sales=pd.read_csv(sales_path, names=['sqft', 'nbedrooms', 'price'])
sales['w0'] = np.ones(sales.shape[0], dtype=int)

#### OLS MATRIX SOLUTION

In [281]:
# construct matrixes
X = np.asarray(sales.loc[:, ['w0', 'sqft', 'nbedrooms']])
Y = np.asarray(sales.price)

# Y_pred = W * X => W = (Xt * X)^-1 * Xt * Y
X_inv = np.linalg.inv(X.T @ X)
W_opt = X_inv @ X.T @ Y
W_opt

array([89597.9095428 ,   139.21067402, -8738.01911233])

#### GRADIENT DESCENT

In [319]:
def gradient(w, x, y):
    y_pred = (w * x).sum(axis=1)
    loss = np.square(y - y_pred).mean()
    derivatives = np.hstack([
        -2 * ( (y - y_pred) * x.w0 ).sum(),
        -2 * ( (y - y_pred) * x.sqft ).sum(),
        -2 * ( (y - y_pred) * x.nbedrooms ).sum()
    ])
    return loss, derivatives

def get_loss(w, x, y):
    y_pred = (w * x).sum(axis=1)
    return np.square(y - y_pred).mean()
                   
def optimal_lr(w, x, y):
    r = y - (w * x).sum(axis=1)
    z = x * (gradient(w, x, y))[0]
    return (r * z.sum(axis=1)).sum() / (z * z).sum().sum()

In [320]:
get_loss(w, sales.loc[:, ['w0', 'sqft', 'nbedrooms']], sales.price)

4086560111.243777

In [321]:
gradient(w, sales.loc[:, ['w0', 'sqft', 'nbedrooms']], sales.price)

(4086560111.243777, array([-63.06768956,  77.09485523,  19.17805223]))

In [297]:
%%time
import scipy

w = np.array([0, 0, 0])

fd = lambda lr : get_loss(w - lr * grad, sales.loc[:, ['w0', 'sqft', 'nbedrooms']], sales.price)
prev_loss = 0
loss = 100
counter = 0

while abs(loss - prev_loss) > 0.1: # the stopping rule
    prev_loss = loss
    loss, grad = gradient(w, sales.loc[:, ['w0', 'sqft', 'nbedrooms']], sales.price)
    alpha = scipy.optimize.minimize_scalar(fd) # suboptimal problem aka the fastest descent method
    w = w - alpha.x * grad
    
    counter += 1
    if counter % 100 == 0:
        print(f'epoch = {counter},  derives = {grad}, loss = {np.sqrt(loss)}, w = {w}')


epoch = 100,  derives = [-1.81940067e+05 -1.48876971e+02  5.49750103e+04], loss = 64575.62888300343, w = [50766.12185906   139.62802117  2995.44532457]
epoch = 200,  derives = [-8.50780639e+04 -7.93503454e+01  2.57071742e+04], loss = 64068.77587896599, w = [71367.39754417   139.3747379  -3229.46348128]
epoch = 300,  derives = [-4.16953602e+04 -3.79389360e+01  1.25986649e+04], loss = 63960.479410383516, w = [80669.44068363   139.29381753 -6040.18114186]
epoch = 400,  derives = [-1.69496675e+04 -1.16685674e+01  5.12151533e+03], loss = 63931.872898228205, w = [86005.849162     139.25894882 -7652.63736895]
epoch = 500,  derives = [-6.57198338e+03 -4.96798079e+00  1.98579128e+03], loss = 63927.0598972748, w = [88199.37297906   139.22735006 -8315.43544702]
epoch = 600,  derives = [-2.12700377e+03 -1.66211638e+00  6.42695627e+02], loss = 63926.29745832384, w = [89144.69179293   139.21584961 -8601.07423553]
epoch = 700,  derives = [-8.11107046e+02 -6.51554104e-01  2.45084141e+02], loss = 63926

#### SKLEARN REGRESSOR

In [87]:
from sklearn.metrics import r2_score

In [88]:
lin_reg = LinearRegression(fit_intercept=False)
lin_reg.fit(X, Y)
lin_reg.coef_
r_score = lin_reg.score(X, Y)

r2 = 1 - ((Y - (X*lin_reg.coef_).sum(axis=1) )**2).sum() / ((Y - Y.mean())**2).sum()
r2_score_lib = r2_score(Y, (X*lin_reg.coef_).sum(axis=1))
lin_reg.coef_, r_score, r2, r2_score_lib

(array([89597.9095428 ,   139.21067402, -8738.01911233]),
 0.7329450180289142,
 0.7329450180289142,
 0.7329450180289142)

#### GRADIENT DESCENT JAX

In [394]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

In [504]:
def get_loss(w, x, y):
    y_pred = jnp.dot(x, w)
    return jnp.sqrt(jnp.square(y - y_pred).mean())

loss_grad = grad(fun=get_loss, argnums=0)

In [505]:
W_jnp = jnp.array([0, 0, 0], dtype=jnp.float32)
X_jnp = jnp.asarray(sales.loc[:, ['w0', 'sqft', 'nbedrooms']], dtype=jnp.float32)
Y_jnp = jnp.asarray(sales.price,  dtype=jnp.float32)

In [518]:
%%time
import scipy

prev_loss = 0
loss = 100
counter = 0

W_jnp = jnp.array([-100, -100, -100], dtype=jnp.float32)
X_jnp = jnp.asarray(sales.loc[:, ['w0', 'sqft', 'nbedrooms']])
Y_jnp = jnp.asarray(sales.price)

fd = lambda lr : get_loss(W_jnp - lr * grad_val,  X_jnp, Y_jnp)
fd_grad = grad(fd)

tol = 1e-7
#alpha = 1e-3
norm = 1
alpha = 1.

while norm > tol: # the stopping rule
    norm = jnp.square(W_jnp-W_true).mean()
    prev_loss = loss
    loss = get_loss(W_jnp, X_jnp, Y_jnp)
    grad_val = loss_grad(W_jnp, X_jnp, Y_jnp)
    #alpha = scipy.optimize.minimize_scalar(fd) # suboptimal problem aka the fastest descent method
    
    if counter < 50:
        alpha = scipy.optimize.minimize_scalar(fd).x # suboptimal problem aka the fastest descent method
    else:
        alpha = 10 * jnp.exp(-counter * 1e-5)
    
    # alpha = 1e-7
    # for i in range(50):
    #     a_grad = fd_grad(alpha)
        #print(f'alpha = {alpha}, loss = {fd(alpha)}, grad = {a_grad}')
        #print(len(str(int(a_grad))))
        #a_grad = a_grad * 10 ** (-len(str(int(a_grad)))) * 1e-1
        #alpha = alpha - a_grad
    
    counter += 1
    if counter % 1000 == 0:
        print(f'epoch = {counter},  derives = {grad_val}, alpha = {alpha}, loss = {loss}, w = {W_jnp}')
        
    W_jnp = W_jnp - alpha * grad_val * jnp.exp(0)
        
        
print(f'estimated = {W_jnp}, true = {W_true}')

epoch = 1000,  derives = [-9.3095273e-01 -2.1496101e+03 -3.1051648e+00], alpha = 9.90059757232666, loss = 40289852.0, w = [   -88.09247  -18577.254       -91.937675]
epoch = 2000,  derives = [-9.3096262e-01 -2.1496101e+03 -3.1051812e+00], alpha = 9.802084922790527, loss = 38621856.0, w = [   -78.83465 -17801.328      -75.25568]
epoch = 3000,  derives = [-9.3097091e-01 -2.1496101e+03 -3.1051955e+00], alpha = 9.70455265045166, loss = 37357656.0, w = [   -70.782486 -17213.246       -60.81272 ]
epoch = 4000,  derives = [-9.3097818e-01 -2.1496099e+03 -3.1052077e+00], alpha = 9.607991218566895, loss = 36294008.0, w = [   -63.40567  -16718.459       -47.616276]
epoch = 5000,  derives = [-9.3098480e-01 -2.1496096e+03 -3.1052191e+00], alpha = 9.512389183044434, loss = 35355996.0, w = [   -56.48243 -16282.117      -35.25466]
epoch = 6000,  derives = [-9.309910e-01 -2.149609e+03 -3.105229e+00], alpha = 9.417739868164062, loss = 34504340.0, w = [   -49.892902 -15885.943       -23.506308]
epoch = 7

KeyboardInterrupt: 

In [485]:
def get_loss(w, x, y):
    y_pred = (w * x).sum(axis=1)
    return jnp.square(y - y_pred).mean()

loss_grad = grad(fun=get_loss, argnums=0)

In [495]:
%%time
import scipy

key = random.key(0)
X_jnp = random.normal(key, shape=(10, 3), dtype=jnp.float32)
W_true = random.choice(key, a=np.array([-2., -1., 1., 2.]), shape=(3,))
Y_jnp = jnp.dot(X_jnp, W_true.T)

W_jnp = jnp.array([0, 0, 0], dtype=jnp.float32)

prev_loss = 0
loss = 100
counter = 0

fd = lambda lr : get_loss(W_jnp - lr * grad_val,  X_jnp, Y_jnp)
fd_grad = grad(fd)

tol = 1e-10
#alpha = 1e-3
norm = 1

while norm > tol: # the stopping rule
    norm = jnp.square(W_jnp-W_true).mean()
    prev_loss = loss
    loss = get_loss(W_jnp, X_jnp, Y_jnp)
    grad_val = loss_grad(W_jnp, X_jnp, Y_jnp)
    #alpha = scipy.optimize.minimize_scalar(fd) # suboptimal problem aka the fastest descent method
    
    alpha = 1.
    for i in range(10):
        a_grad = fd_grad(alpha)
        alpha = alpha - a_grad*1e-2
        #print(f'alpha = {alpha}, loss = {fd(alpha)}, grad = {a_grad}')
    
    counter += 1
    if counter % 1 == 0:
        print(f'epoch = {counter},  derives = {grad_val}, alpha = {alpha}, loss = {loss}, w = {W_jnp}')
        
    W_jnp = W_jnp - alpha * grad_val * jnp.exp(-counter/100)
        
        
print(f'estimated = {W_jnp}, true = {W_true}')

epoch = 1,  derives = [ 1.4601613 -3.9329948  3.3388686], alpha = 0.45790791511535645, loss = 7.0625901222229, w = [0. 0. 0.]
epoch = 2,  derives = [ 0.7447395   0.19095409 -0.01559857], alpha = 0.9903147220611572, loss = 0.48153382539749146, w = [-0.66196656  1.7830297  -1.5136817 ]
epoch = 3,  derives = [ 0.03411987 -0.58463913 -0.4215534 ], alpha = 0.9536437392234802, loss = 0.23318345844745636, w = [-1.3848891  1.5976696 -1.49854  ]
epoch = 4,  derives = [0.507192   0.61516744 0.16041796], alpha = 0.9192235469818115, loss = 0.1819574534893036, w = [-1.4164656  2.1387293 -1.1084095]
epoch = 5,  derives = [-0.22898938 -0.8513918  -0.29063815], alpha = 0.8879203796386719, loss = 0.19304361939430237, w = [-1.8644077  1.5954256 -1.2500875]
epoch = 6,  derives = [0.43204594 0.89476156 0.15340976], alpha = 0.867331862449646, loss = 0.21143034100532532, w = [-1.6709995  2.3145247 -1.0046098]
epoch = 7,  derives = [-0.29770172 -0.9718285  -0.17948818], alpha = 0.8597367405891418, loss = 0.2